# Message simulation for I2V Connectivity

The following corresponds to a test for simulating impact of vehicle-to-infrastructure (V2I) communication in traffic systems. 

## General description

For this application it is considered the simulation of microscopic traffic models where longitudinal position follow a specified behivior defined by two main components. The *car following* behavior describes the behavior of the vehicle in its longitudinal dynamics while the *lane change* behavior describes the behavior in the lateral position. 

In order to modify traffic behavior for a condition, the system is modeled via traffic model where V2I messages modify vehicle speed or lateral position   

### Car following behavior

For the sake of clarity, the following corresponds to the notations for variable description in the model. It is considered the vehicle position of a vehicle as $x_n$ and the headway space between a vehicle and its leader as $s_n = x_{n-1}-x_{n}$. The vehicle's speed and acceleration are defined as $v_n$,$a_n$ respectively.

For a determined vehicle in the network the longitudinal dynamics are determined by the acceleration behavior. In this case it is considred Tampere's Law. 

$$ 
a_n(t+T_n) = \min \left(c_{1,n-1}\Delta v_{n-1,n} + 
c_{2}\left(\Delta x_{n,n-1} - \left(s_0+\tau v_n(t)\right)\right),
c_{3}\left(v^\star(t) - v_n(t)\right)\right)
$$

One of the main features of this model is the adaptability to a specific speed condition, while preserving properties of the traffic such as the car following behavior in congestion situation. This feature makes it possible to trace features in the fundamental diagram. 

*To implement the model a `class` object called `Tampere` has been implemented. The class intends to describe the full behavior of the vehicle.* 

#### Parameters 

So far parameters in the model have been fixed although random scenarios can be also considered.

| Parameter     | Value     | Units |
:--------------:|:---------:|:------:
$$c_1,c_2,c_3$$ | 0.5       |
$$\tau$$        | $$\frac{1}{wk_x}$$ | [s]
$$w$$           | $$6.25$$  | [m/s]
$$k_x$$         | $$0.16$$  | [veh/km]
$$u_i$$         | $$25$$    | [m/s]

## Simulation Usage 

Please follow step by step variable definitions for more detail into simulations


In [1]:
from carfollow import Tampere, W_I, U_I, K_X, A_MIN, A_MAX
from support import speed_pulse
from messages import Msg1, Msg2
import numpy as np
import pandas as pd
import datetime as dt
import csv 

from plottools import plot_single_trace, plot_xva, plot_histogram, plot_multiple_trajectories
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, export_png
from bokeh.layouts import row, column

output_notebook()

Loading BokehJS ...

#### Hyper parameter variation for sensitivity analysis

* `MPR`: Market penetration rate [0,1]
* `Q_PERC`: Percentage of maximum flow (0,1]
* `MIN_DIST`: Minium distance for message broadcasting [4000,14000]

In [2]:
# Parameters
MPR = 0
MIN_DIST = 5000
Q_PERC = 0.3


## Input definitions 

Main input definitions for the simulation such as the `simulation time`, `flow`, `distance to congestion`, `congestion length`, `perception radious`

**Important note**: To run simulations be sure to start always from the beginning

### <span style="color:red">Define parameters below </span>

<span style= "color:red">See comments for more details </span>

* simulation time: `time`
* works position: `X_CONGESTION`
* works length: `L_CONGESTION`
* radious of messages: `PERCEP_RADIOUS`

In [3]:
# Constant values
N = 100  # Number of vehicles to simulate
T_TOTAL = 960  # Simulation time [s]
time = np.arange(T_TOTAL)  # Time vector

# Traffic characteristics
X_CONGESTION = 15000  # Position of congestion in space [m]
L_CONGESTION = 1500  # Approximate congestion length in space [m]

# Messages for V2V
SPEED_REDUCTION = 5.5  # Amount of speed reduction [m/s]
PERCEP_RADIOUS = 3000  # Radious of perception of the broadcasted messages [m]

## Vehicle definitions 

In the following cell content regarding the vehicle will be defined in particular the `flow`. The current capacity value is: 



### <span style="color:red">Define parameters below </span>

<span style= "color:red">See comments for more details </span>

* Amount of flow w.r.t to capacity: `Q_PERC`

In [4]:
# Capacity
C = (U_I * W_I * K_X) / (W_I + U_I)
# print(f"Capacity value per lane: {C*3600} [veh/h]")

# Vehicle Initial position / speed
TF = C * Q_PERC

In [5]:
# Vehicle initializer
X0 = np.flip(np.arange(0, N) * (W_I + U_I) / (W_I * K_X) * 1 / Q_PERC)
V0 = np.ones(N) * U_I
A0 = np.zeros(N)

veh_list = []

np.random.seed(42)  # Reproducibility
ID_CAV = np.random.randint(1, N - 1, int(N * MPR))  # Id Connected Vehicles
D_CLASS = {k: "CAV" for k in ID_CAV}
V_CLASS = [D_CLASS.get(i, "HDV") for i in range(N)]  # All vehicle types

# Initializing vehicles
Tampere.reset()
for x0, v0, vtype in zip(X0, V0, V_CLASS):
    veh_list.append(Tampere(x0=x0, v0=v0, l0=0, veh_type=vtype))

# Setting leader for vehicle i
for i in range(1, N):
    veh_list[i].set_leader(veh_list[i - 1])

ID_CAVN = [i for i, j in enumerate(V_CLASS) if j == "CAV"]

## Message characteristics 

The following cell computes the distances at which the messages are going to be broadcasted 

In [6]:
# Scenario conditions
D_ACCEPT = X_CONGESTION - 1000  # Broad casting messages @ 14Km
D_ACCEPT = D_ACCEPT - np.random.exponential(PERCEP_RADIOUS, N * 1000)
D_ACCEPT = D_ACCEPT[(D_ACCEPT > MIN_DIST) & (D_ACCEPT < X_CONGESTION)]
D_ACCEPT = np.random.choice(D_ACCEPT, N)

# accept_dist = plot_histogram(D_ACCEPT,"Acceptane Distance [Km]")
# show(accept_dist)

## Message information 

The following is just to illustrate the message generation and definitions for the simulation 

In [7]:
# Road works speed profile
def lead_spd(x):
    """  Leader's function to control speed drop in space 
         Speed Drop: 20 m/s 
         Position: 15 Km
         Duration: 20 Km
    """
    return speed_pulse(x, drop=20, delay=X_CONGESTION, duration=L_CONGESTION)

# x_t = np.linspace(0, 20000, 20000)
# v_t = lead_spd(x_t)
# leader_xt = plot_single_trace(
#      x_t, v_t, "Leaders' speed", "Space [m]", "Speed [m/s]"
# )
# show(leader_xt)

## Main simulation

The following code defines the simulation that is run. Execute the code directly in case you want to modify the message change `Msg1` by `Msg2`

In [8]:
# Dynamical evalution
X = X0
V = V0
A = A0

send_message = Msg2  # Msg2 # Defines the type of message to be send

d_accept = X_CONGESTION - 1000
msg_fix = send_message(d_accept)

for t in time:
    for veh in veh_list:
        if veh.type == "CAV" and not veh.acc:
            d_accept = D_ACCEPT[veh.idx]
            msg = send_message(d_accept)
            veh.register_control_speed(msg)
        elif veh.type == "HDV" and not veh.acc:
            veh.register_control_speed(msg_fix)

        veh.step_evolution(control=lead_spd)

    V = np.vstack((V, np.array([veh.v for veh in veh_list])))
    X = np.vstack((X, np.array([veh.x for veh in veh_list])))
    A = np.vstack((A, np.array([veh.a for veh in veh_list])))

V = V[1:, :]
X = X[1:, :]
A = A[1:, :]

# Indicators 

The following produces the total travel time for vehicles

In [ ]:
df_x = pd.DataFrame(X)
now = dt.datetime.now()
delta = dt.timedelta(seconds=1)
time_vector = [now+n*delta for n in range(T_TOTAL)]
df_x.index = time_vector 

def find_travel_time(df):
    time_entry  = min(df[df>df_x.iloc[0,0]].index) # minimum time after leader entrance 
    time_exit = max(df[df<X_CONGESTION+L_CONGESTION].index) # maximum time before exit
    travel_time = time_exit-time_entry
    return travel_time.total_seconds()

ttt = df_x.apply(find_travel_time)

tt_mean,tt_var = np.mean(ttt),np.std(ttt) 
ttt = sum(ttt)
# tthist = plot_histogram(ttt)
# show(tthist)

# Plots and results 

The following are extracts of some plots and results to be analyzed 

In [ ]:
# Creating plots
x_t = time
v_t = V[:, 0]
leader_vt = plot_single_trace(x_t, v_t, "Leaders' speed", "Time [s]", "Speed [m/s]")
x_t = time
a_t = A[:, 0]
leader_at = plot_single_trace(x_t, a_t, "Leaders' acceleration", "Time [m]", "Acceleration [m/s²]")
zooms = ((MIN_DIST, X_CONGESTION + L_CONGESTION), (-1, U_I + 1), (A_MIN - 0.5, A_MAX + 0.5))
titles = (
    f"X-Time MPR={MPR*100}% F={TF*3600}[veh/h] D={MIN_DIST}[m]",
    f"V-Time MPR={MPR*100}% F={TF*3600}[veh/h] D={MIN_DIST}[m]",
    f"X-Time MPR={MPR*100}% F={TF*3600}[veh/h] D={MIN_DIST}[m]"
  )
pos, spd, acc = plot_xva(time, X, V, A, y_range=zooms, titles=titles)
poswoz = plot_multiple_trajectories(time, X, V, titles[0], "Time [secs]", "Position [m]")
# show(leader_at)

# Exporting files

The following are extracts of some plots and results to be analyzed 

In [ ]:
# Writting CSV File for Phem 

df_X = pd.DataFrame(X)
df_X = df_X.melt(var_name="id",value_name="x")
df_X = df_X.reset_index()
df_X = df_X.rename(columns={'index': 'time',"id":"vehicle number"})
df_X["y"] = 0
df_v = pd.DataFrame(V)
df_V = df_v.melt(var_name="id",value_name="v")
df_X["speed"]=df_V["v"]*3.6 
df_X["road inclination"] = 0 
df_X["vehicle type"] = 100
df_X["segment number"] = 0
correct_order = [0,2,3,1,4,5,6,7]
df_X = df_X[df_X.columns[correct_order]]
data = "data/csv/"
filename = f"phem_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.csv"
df_X.to_csv(data+filename)

In [ ]:
# Writting trajectories file
data = "data/"
filename = f"img/cases/postition/pos_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.png"
export_png(pos, filename=data + filename)
filename = f"img/cases/postition/poswoz_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.png"
export_png(poswoz, filename=data + filename)
print(f"File: {filename} has been saved")
filename = f"img/cases/speed/spd_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.png"
export_png(spd, filename=data + filename)
print(f"File: {filename} has been saved")
filename = f"img/cases/acceleration/acc_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.png"
export_png(acc, filename=data + filename)
print(f"File: {filename} has been saved")

In [ ]:
# Writting indicators file 

csvTitle = ['mpr','q','distance','meanTT','stdTT','totalTT']
csvData = [MPR,TF,MIN_DIST,tt_mean,tt_var,ttt]
print(f"MPR:{MPR},Q:{TF},D:{MIN_DIST}")

try:
    with open('data/Indicators.csv','a') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(csvData)
except FileNotFoundError:
    with open('data/Indicators.csv','w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(csvTitle)
        writer.writerow(csvData)
        

A. Ladino